In [3]:
import spacy
import language_tool_python
import requests.exceptions

def initialize_language_tool():
    try:
        return language_tool_python.LanguageTool('en-US')
    except requests.exceptions.RequestException as e:
        print(f"Error initializing LanguageTool: {e}")
        return None

# Load spaCy model and add the 'sentencizer' component
nlp = spacy.load("en_core_web_sm")
sentencizer = nlp.add_pipe('sentencizer')

tool = initialize_language_tool()

def process_and_grade_essay(essay):
    # Step 1: Process the essay using spaCy
    doc = nlp(essay)

    # Step 2: Extract faults, grammar mistakes, and suggestions
    faults = []

    if tool:
        grammar_mistakes = tool.check(essay)
        corrected_essay = tool.correct(essay)

        # Provide more detailed feedback for each grammar mistake
        detailed_feedback = []
        for match in grammar_mistakes:
          if hasattr(match, 'fromx') and hasattr(match, 'tox'):
            # Use match.fromx and match.tox for character indices
            detailed_feedback.append(f"Error from character {match.fromx} to {match.tox}: {match.message}")
          elif hasattr(match, 'fromy') and hasattr(match, 'toy'):
            # Use match.fromy and match.toy for line and column numbers
            detailed_feedback.append(f"Error at line {match.fromy}, column {match.fromycol}: {match.message}")
          else:
            detailed_feedback.append(f"Error: {match.message}")


        # Include detailed feedback in the result
        result = {
            'grammar_mistakes': grammar_mistakes,
            'detailed_feedback': detailed_feedback,
            'corrected_essay': corrected_essay
        }
    else:
        # Provide a default feedback if LanguageTool initialization fails
        result = {
            'grammar_mistakes': [],
            'detailed_feedback': ["Error initializing LanguageTool. Please check your internet connection."],
            'corrected_essay': essay
        }

    # Step 3: Provide feedback on essay structure and content
    has_introduction = any(sent.text.lower().startswith('introduction') for sent in doc.sents)
    has_body = any(sent.text.lower().startswith('body') for sent in doc.sents)
    has_conclusion = any(sent.text.lower().startswith('conclusion') for sent in doc.sents)

    if not has_introduction:
        faults.append("The essay lacks an introduction.")

    if not has_body:
        faults.append("The essay lacks a body section.")

    if not has_conclusion:
        faults.append("The essay lacks a conclusion.")

    # Step 4: Provide overall feedback
    feedback = "Overall, your essay could benefit from the following improvements:\n"
    feedback += "\n".join(faults)

    # Include overall feedback in the result
    result['feedback'] = feedback

    # Step 5: Return the result
    return result

# Example usage:
user_essay = '''
Introduction:
In today's fast-paced world, technology plays a pivotal role in shaping our lives. From communication to education, technology has revolutionized the way we interact with the world. In this essay, we will explore the impact of technology on society, focusing on its benefits and challenges.

Body:
Technology has transformed communication, making it easier for people to connect across the globe. Social media platforms enable instant communication and the sharing of ideas. Additionally, advancements in education technology have expanded access to learning resources, breaking down geographical barriers.

However, the widespread use of technology also presents challenges. One major concern is the potential for privacy breaches and data security issues. As we embrace the convenience of technology, it is crucial to address these concerns to safeguard our personal information.

Conclusion:
In conclusion, technology has become an integral part of our daily lives, offering numerous advantages. Despite the benefits, it is essential to navigate the challenges responsibly. By understanding and mitigating the risks associated with technology, we can harness its potential for positive change.
'''
result = process_and_grade_essay(user_essay)

print("Grammar Mistakes:", result['grammar_mistakes'])
print("\nDetailed Feedback:", result['detailed_feedback'])
print("\nFeedback:", result['feedback'])
print("\nCorrected Essay:", result['corrected_essay'])


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpjzwjuho7.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


Grammar Mistakes: []

Detailed Feedback: []

Feedback: Overall, your essay could benefit from the following improvements:
The essay lacks an introduction.

Corrected Essay: 
Introduction:
In today's fast-paced world, technology plays a pivotal role in shaping our lives. From communication to education, technology has revolutionized the way we interact with the world. In this essay, we will explore the impact of technology on society, focusing on its benefits and challenges.

Body:
Technology has transformed communication, making it easier for people to connect across the globe. Social media platforms enable instant communication and the sharing of ideas. Additionally, advancements in education technology have expanded access to learning resources, breaking down geographical barriers.

However, the widespread use of technology also presents challenges. One major concern is the potential for privacy breaches and data security issues. As we embrace the convenience of technology, it is cru

In [2]:
pip install language-tool-python

In [ ]:
pip install spacy language_tool_python
